PILAR ARIAS, YEDRA MARTÍN, PATRICIA MEDIAVILLA, AMAYA SARASA

<GRUPO 9>

#INTRODUCCIÓN

Esta es la primera práctica del primer laboratorio. Dada una base de datos en formato .csv, se nos pide contestar a una serie de cuestiones. Para ello, lo primero que debemos hacer es cargar y leer la base de datos:

In [1]:
#importamos la base de datos
import pandas as pd #pandas
df=pd.read_csv('airbnb.csv',encoding='latin-1')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


Se observa que se ha cargado de manera correcta. 

Importamos así también todas las librerías necesarias para ejecutar esta práctica. 

In [2]:
#importamos todas las librerías necesarias: 
import sklearn #scikit-learn
import matplotlib.pyplot as plt #gráfica Matplotlib
import numpy as np #NumPy

Según se ha estudiado en clase, encontramos dos métodos para afrontarlo. Nos decantaremos por CRISP-DM. Las siglas para: CRoss-Industry Standard Process for Data Mining. 

Es un modelo de proceso de minería de datos que se utiliza en la industria. Es un marco de trabajo que define un proceso estandarizado para llevar a cabo proyectos de minería de datos, desde la definición del problema hasta la implementación y evaluación del resultado. Los seis pasos que componen Crisp-DM son los siguientes:

- Comprensión de los negocios y definición del problema: comprender la naturaleza del problema y los objetivos de negocio.

- Comprensión de los datos: recopilación inicial, descripción y exploración  de datos.

- Preparación de los datos: Selección, limpieza, construcción, integración, formateo de los datos.

- Modelado: aplicar las técnicas de minería de datos a los dataset.

- Evaluación: determinar si los resultados son útiles a las necesidades del negocio.

- Despliegue: explotar utilidad de los modelos, integrándolos en las tareas de toma de decisiones de la organización -> Call to Action. 


#EJERCICIO 1

## Comprensión del negocio y definición del problema

El barrio de Williamsburg es uno de los barrios de moda hoy en día. Desde 2005 ha pasado de ser un distrito donde principalmente se encontraban fabricas a acoger una gran cantidad de gente joven y nuevos negocios. ¿Teniendo en cuenta los datos proporcionados que posición en términos de popularidad/calidad se puede decir que ostenta el barrio con respecto a los demás que forma New York?

Para medir la popularidad/calidad del barrio, tendremos primero en cuenta el número de apartamentos de calidad según los usuarios. Para ello, lo primero será clasificar los apartamentos como “Muy Malos”, “Malos”, “Regulares”, “Buenos” y “Muy Buenos”. Para ello primero haremos un par de transformaciones de los datos. La columna de reseñas por mes, se dividirán en 3 rangos iguales, después dependiendo del valor concreto que tenga cada apartamento se le asignarán las etiquetas: “Baja”, “Media” y ”Alta”. Para la columna que indica el número de días que está disponible asignaremos las etiquetas de menor valor a mayor de la siguiente manera: “Poco disponible”, “Disponible normalmente” y “Altamente disponible”. Finalmente crearemos una columna para clasificar los apartamentos, teniendo en cuenta una serie de reglas, que se explicarán más adelante. 

## Comprensión de los datos

La base de datos que utilizaremos consta de 48895 observaciones. En cada observación recopilamos los siguientes atributos: 
- id: id del apartamento en cuestión. 
- name: nombre del apartamento
- host_id: id del anfitrión 
- host_name: nombre del anfitrión 
- neighbourhood_group: estado al que pertenece
- neighbourhood: barrio al que pertenece 
- latitude: coordenadas de latitud
- longitude: coordenadas de longitud 
- room_type: tipo de habitación 
- price: precio en dólares
- minimum_nights: noches mínimas que te puedes hospedar
- number_of_reviews: número de reseñas 
- last_review: última reseña 
- reviews_per_month: reseñas por mes 
- calculated_host_listings_count: ??? 
- availability_365: días disponibles por año. 


TENEMOS QUE ESTUDIAR ESTO: datos redundantes o anómalos.

## Preparación de los datos

Primero debemos tratar la base de datos para así estudiar qué hacer con los datos NaN que encontraremos. 

Analizamos cuántos datos NaN hay en la columna 'reviews_per_month', ya que la trataremos posteriormente, y deberemos conocer el contenido de esta. 


In [3]:
reviews_per_month_nan = df['reviews_per_month'].isna().sum()
print(reviews_per_month_nan)

10052


Ahora bien, vemos que hay 10052 datos NaN en la columna. Observemos cuántos datos hay en total para barajar las distintas opciones de qué hacer con estos datos NaN. Si es poca cantidad comparado con la cantidad de observaciones en total, podremos eliminar directamente esas observaciones, sin embargo, si suponen gran cantidad, podríamos buscar otras opciones, como rellenar con la media para así no afectar al equilibrio del resto de datos. 

Es verdad que como hablamos de reseñas por mes, sería factible convertir esos NaN en 0, para que así a la hora de categorizar la columna, lo detecte como Bajo. 

¿Por qué rellenamos con 0's? Encontramos un NaN en la columna de 'reviews_per_month', cuando en la columna de 'number_of_reviews' aparece un 0, o un NaN directamente, es decir, que anteriormente no se ha hecho ninguna reseña, por lo que obviamente, en el último mes, tampoco. No deberíamos rellenar con la media, ya que ahí estamos aportando una cierta cantidad de reseñas que no se han hecho. 

In [4]:
num_observations = df.shape[0]
print(num_observations)

48895


Supone un 20% de los datos totales. No resulta útil borrarlos, si no que deberíamos rellenar con 0 como hemos explicado anteriormente. 

In [5]:
df['reviews_per_month'].fillna(0, inplace=True)

Comprobamos por tanto que en esa columna ya no nos quede ningún NaN

In [6]:
reviews_per_month_nan = df['reviews_per_month'].isna().sum()
print(reviews_per_month_nan)

0


Ya nos hemos asegurado que no nos quedad ningún dato NaN, así no encontraremos ningún error a la hora de categorizar la columna, que es el siguiente paso. 

Lo categorizamos en tres: Baja, Media y Alta. Para ello tomaremos el valor del menor y mayor dato, y etiquetando así cada clase, gracias a la función pd.cut, asignaremos cada valor numérico correspondiente. 

In [7]:
# Número de intervalos
num_intervalos = 3

# Lista de intervalos
intervalos = np.linspace(df['reviews_per_month'].min()-0.001, df['reviews_per_month'].max(), num_intervalos+1)

# Creamos las etiquetas 
etiquetas = ['Baja', 'Media', 'Alta']

# Asigna una categoría a cada intervalo
df['reviews_per_month'] = pd.cut(df['reviews_per_month'], intervalos, labels=etiquetas)

En la lista de intervalos se ha restado -0.001, ya que al tomar el rango mínimo como 0, no incluía ese valor y detectaba aquellos valores 0.00 como NaN a la hora de categorizar. 

Por otro lado, vamos a clasificar los alquileres en función de los días que están disponibles. 

Para ello, siguiendo el patrón anterior, calculamos si hay NaN en la columna correspondiente: 


In [8]:
disponibilidad_nan = df['availability_365'].isna().sum()
print(disponibilidad_nan)

0


No hay ningún NaN en dicha columna, por lo que no hace falta tratarlos. En el caso de encontrarlos, se tendría que estudiar qué hacer con este conjunto de datos de manera independiente. 

Calculemos los intervalos de los rangos. 

In [9]:
#Definimos los intervalos
n_inter = 3

# Creamos una lista de intervalos
interv = np.linspace(df['availability_365'].min()-0.001, df['availability_365'].max(), n_inter+1)

# Creamos las etiquetas 
labels = ['Poco disponible', 'Disponible normalmente', 'Altamente disponible']

# Asigna una categoría a cada intervalo
df['availability_365'] = pd.cut(df['availability_365'], interv, labels=labels)

Imprimimos 5 observaciones del nuevo atributo categorizado para observar si se ha realizado de manera correcta. 

In [10]:
print(df['availability_365'].head())

0      Altamente disponible
1      Altamente disponible
2      Altamente disponible
3    Disponible normalmente
4           Poco disponible
Name: availability_365, dtype: category
Categories (3, object): ['Poco disponible' < 'Disponible normalmente' < 'Altamente disponible']


Finalmente, añadimos la nueva columna de clasificación, y la rellenamos con datos NaN, para así más adelante rellenarlo según las reglas definidas en el enunciado.

In [11]:
df.loc[:,'classification']=np.nan

Imprimimos las 5 primeras observaciones con cada atributo para comprobar que se ha añadidio de manera correcta

In [12]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,classification
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,Baja,6,Altamente disponible,NaN
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,Baja,2,Altamente disponible,NaN
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,Baja,1,Altamente disponible,NaN
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,Baja,1,Disponible normalmente,NaN
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,Baja,1,Poco disponible,NaN


Vamos a clasificar los datos según la siguente tabla

 Poco disponible | Disponible normalmente | Altamente disponible
-----:|:--------:| -------
Bueno | Regular | Malo
Bueno | Regular | Malo
Muy bueno | Bueno | Muy Malo



Donde cada fila depende del valor de la reseña en Baja, Media y Alta respectivamente

In [13]:

filas=len(df.axes[0])
for i in range (filas):
  if df['availability_365'][i]=='Poco disponible':
    if df['reviews_per_month'][i]=='Alta':
      df['classification'][i]='Muy bueno'
    else:
      df['classification'][i]='Bueno'
  elif df['availability_365'][i]=='Disponible normalmente':
    if df['reviews_per_month'][i]=='Alta':
      df['classification'][i]='Bueno'
    else:
      df['classification'][i]='Regular'
  else:
    if df['reviews_per_month'][i]=='Alta':
      df['classification'][i]='Muy malo'
    else:
      df['classification'][i]='Malo'


<ipython-input-13-40d0a61399f8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['classification'][i]='Malo'
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Comprobemos que no quede ningun NaN. 

In [14]:
classif_nan = df['classification'].isna().sum()
print(classif_nan)

0


Todos los datos de la nueva columna han sido modificados, por lo que ya tenemos clasificados los alquieleres. 

Estudiaremos cuantos apartamentos pertenecen a cada clase. 



In [15]:
cant_bueno = df['classification'].value_counts() 
print(cant_bueno)


Bueno       30637
Malo        11474
Regular      6783
Muy malo        1
Name: classification, dtype: int64


Hay un total de 30 637 alquileres buenos, 11 474 malos y solamente 1 muy malo, por lo que podemos considerar que de los alquileres en esta página predominan los buenos. 

## Modelado


Una vez hemos preparado los datos tal y como deseamos, procedemos a modelar y estudiar el objetivo. Vamos a ver cuántos barrios hay, y cuántos apartamentos encontramos en los más populares. 

In [16]:
df['neighbourhood'].value_counts()


Williamsburg          3920
Bedford-Stuyvesant    3714
Harlem                2658
Bushwick              2465
Upper West Side       1971
                      ... 
Fort Wadsworth           1
Richmondtown             1
New Dorp                 1
Rossville                1
Willowbrook              1
Name: neighbourhood, Length: 221, dtype: int64

Primero observamos que Williamsburg es donde más apartamentos encontramos.


In [17]:
df.loc[:,'classification_num']=np.nan

In [18]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,classification,classification_num
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,Baja,6,Altamente disponible,Malo,NaN
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,Baja,2,Altamente disponible,Malo,NaN
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,Baja,1,Altamente disponible,Malo,NaN
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,Baja,1,Disponible normalmente,Regular,NaN
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,Baja,1,Poco disponible,Bueno,NaN


In [19]:
filas=len(df.axes[0])
for i in range (filas):
  if df['availability_365'][i]=='Poco disponible':
    if df['reviews_per_month'][i]=='Alta':
      df['classification_num'][i]=1
    else:
      df['classification_num'][i]=0.75
  elif df['availability_365'][i]=='Disponible normalmente':
    if df['reviews_per_month'][i]=='Alta':
      df['classification_num'][i]=0.75
    else:
      df['classification_num'][i]=0.5
  else:
    if df['reviews_per_month'][i]=='Alta':
      df['classification_num'][i]=0
    else:
      df['classification_num'][i]=0.25

<ipython-input-19-812340fe2f91>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['classification_num'][i]=0.25
<ipython-input-19-812340fe2f91>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['classification_num'][i]=0.5
<ipython-input-19-812340fe2f91>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['classification_num'][i]=0.75
<ipython-input-19-812340fe2f91>:15: SettingWithCopyWarning: 
A value is trying to be set on a co

In [20]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,classification,classification_num
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,Baja,6,Altamente disponible,Malo,0.25
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,Baja,2,Altamente disponible,Malo,0.25
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,Baja,1,Altamente disponible,Malo,0.25
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,Baja,1,Disponible normalmente,Regular,0.50
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,Baja,1,Poco disponible,Bueno,0.75


In [21]:
barrios= np.unique(df['neighbourhood'])
num_barrios = len(barrios)
print('Number of classes:', num_barrios)

Number of classes: 221


In [22]:
barrios

array(['Allerton', 'Arden Heights', 'Arrochar', 'Arverne', 'Astoria',
       'Bath Beach', 'Battery Park City', 'Bay Ridge', 'Bay Terrace',
       'Bay Terrace, Staten Island', 'Baychester', 'Bayside', 'Bayswater',
       'Bedford-Stuyvesant', 'Belle Harbor', 'Bellerose', 'Belmont',
       'Bensonhurst', 'Bergen Beach', 'Boerum Hill', 'Borough Park',
       'Breezy Point', 'Briarwood', 'Brighton Beach', 'Bronxdale',
       'Brooklyn Heights', 'Brownsville', "Bull's Head", 'Bushwick',
       'Cambria Heights', 'Canarsie', 'Carroll Gardens', 'Castle Hill',
       'Castleton Corners', 'Chelsea', 'Chinatown', 'City Island',
       'Civic Center', 'Claremont Village', 'Clason Point', 'Clifton',
       'Clinton Hill', 'Co-op City', 'Cobble Hill', 'College Point',
       'Columbia St', 'Concord', 'Concourse', 'Concourse Village',
       'Coney Island', 'Corona', 'Crown Heights', 'Cypress Hills',
       'DUMBO', 'Ditmars Steinway', 'Dongan Hills', 'Douglaston',
       'Downtown Brooklyn', 'Dyk

In [23]:
classification_barrios=np.array(np.zeros(num_barrios))
print(classification_barrios.shape)

(221,)


In [26]:
for i in range(len(barrios)):
  for j in range(filas):
    if barrios[i]==df['neighbourhood'][j]:
      classification_barrios[i]=classification_barrios[i]+df['classification_num'][j]


In [28]:
n=np.array(np.zeros(num_barrios))
for i in range(len(barrios)):
  for j in range(filas):
    if barrios[i]==df['neighbourhood'][j]:
      n[i]=n[i]+1


[4.200e+01 4.000e+00 2.100e+01 7.700e+01 9.000e+02 1.700e+01 7.000e+01
 1.410e+02 6.000e+00 2.000e+00 7.000e+00 3.900e+01 1.700e+01 3.714e+03
 8.000e+00 1.400e+01 2.400e+01 7.500e+01 1.000e+01 1.770e+02 1.360e+02
 3.000e+00 5.600e+01 7.500e+01 1.900e+01 1.540e+02 6.100e+01 6.000e+00
 2.465e+03 2.600e+01 1.470e+02 2.330e+02 9.000e+00 4.000e+00 1.113e+03
 3.680e+02 1.800e+01 5.200e+01 2.800e+01 2.100e+01 1.500e+01 5.720e+02
 2.000e+00 9.900e+01 1.900e+01 4.200e+01 2.600e+01 5.000e+01 3.200e+01
 1.700e+01 6.400e+01 1.564e+03 1.350e+02 3.600e+01 3.090e+02 7.000e+00
 8.000e+00 8.300e+01 1.200e+01 1.850e+02 5.000e+02 1.117e+03 1.000e+01
 2.180e+02 1.853e+03 1.300e+01 1.300e+01 1.100e+01 2.370e+02 3.000e+00
 5.000e+00 2.900e+01 1.200e+01 7.440e+02 6.210e+02 8.000e+01 8.300e+01
 4.260e+02 6.300e+01 1.440e+02 4.890e+02 5.500e+01 1.000e+00 3.200e+01
 5.400e+01 2.470e+02 3.380e+02 3.000e+00 6.000e+00 6.800e+01 1.000e+01
 1.115e+03 3.920e+02 7.000e+00 2.658e+03 1.958e+03 2.700e+01 1.400e+01
 4.000

In [30]:
total=classification_barrios/n
print(total)



[0.54761905 0.625      0.39285714 0.49350649 0.60194444 0.47058824
 0.61428571 0.54964539 0.41666667 0.75       0.53571429 0.48717949
 0.47058824 0.5930937  0.5        0.41071429 0.48958333 0.57666667
 0.575      0.64548023 0.52573529 0.66666667 0.51339286 0.50666667
 0.53947368 0.67207792 0.59016393 0.54166667 0.62474645 0.57692308
 0.43367347 0.65665236 0.52777778 0.5        0.59321653 0.63451087
 0.59722222 0.67307692 0.52678571 0.42857143 0.46666667 0.61975524
 0.25       0.67424242 0.57894737 0.67857143 0.39423077 0.53
 0.53125    0.5        0.4140625  0.62132353 0.52222222 0.59722222
 0.59304207 0.53571429 0.4375     0.71385542 0.52083333 0.52702703
 0.5005     0.62421665 0.5        0.50114679 0.65366972 0.28846154
 0.46153846 0.34090909 0.57489451 0.41666667 0.65       0.53448276
 0.54166667 0.46875    0.62399356 0.58125    0.48192771 0.53403756
 0.58730159 0.57638889 0.62832311 0.53636364 0.25       0.578125
 0.54166667 0.58603239 0.61982249 0.5        0.375      0.59191176
 0.

In [ ]:
#ya lo sabemos de antes, pero por calcularlo. Esto es cuántos apartamentos en el barrio Williamsburg hay, para luego dividirlo para hacer la media
n=0
filas=len(df.axes[0])
for i in range(filas):
  if df['neighbourhood'][i]=='Williamsburg':
    n=n+1
n

3920

In [ ]:
media_val_barrio=0
filas=len(df.axes[0])
for i in range(filas):
  if df['neighbourhood'][i]=='Williamsburg':
    media_val_barrio=media_val_barrio+df['classification_num'][i]
media_val_barrio

2560.5

In [ ]:
media_val_barrio=media_val_barrio/n

In [ ]:
media_val_barrio

0.653188775510204

In [ ]:

if media_val_barrio[i]<0.2:
    popularidad='Muy malo'
elif media_val_barrio[i]<0.4:
    popularidad='Malo'
elif media_val_barrio[i]<0.6:
    popularidad='Regular'
elif media_val_barrio[i]<0.8:
    popularidad='Bueno'
else:
    popularidad='Muy bueno'

p
#0.653188775510204
#Bueno
#Evaluación

Bueno


------------------------------------------------------------------

df['neighbourhood'].value_counts()

##Evaluación


##Despliegue

#EJERCICIO 2

 Explica visualmente como se distribuyen los 5 barrios más populares (esto serán aquellos cuyos apartamentos estén más solicitados en números absolutos. Para ello utiliza un diagrama de burbuja donde el eje de las X indica el barrio y el eje Y la calidad de los apartamentos. Usa todos los diagramas que necesites para llegar a esta conclusión final.

In [ ]:
#Bubble Plot. The size of points changes based on a third variable
# x = barrio (ya tenemos una variable de los barrios)
# y = calidad de los apartamentos (classification)
# z = disponibilidad
x=barrios
plt.scatter(x,y,s)

#EJERCICIO 3

Una vez obtenida la información del apartado anterior, queremos tratar de entender las diferencias de precio entre alquilar un apartamento entero y una habitación privada. Usa los diagramas de cajas donde cada uno de los 5 barrios más populares está representado por una caja y haz una interpretación de los resultados.

#EJERCICIO 4

Por último, se intuye que la tendencia a dejar reseñas en las apps que prestan servicios, ha aumentado en los últimos años. Decide que diagrama es más útil para este caso. Dibújalo y realiza una interpretación del mismo. Tomaremos la fecha de la última reseña como el dato útil para realizar dicho caso.